In [10]:
import os
import pickle
from sklearn.svm import LinearSVC

from packages import load_MTCNN as lmtcnn
from packages import load_face_pkl as pkl


In [12]:
DATA_PATH = 'data/' #data目錄
    
IMG_OUT_PATH = os.path.join(DATA_PATH, "dataset") #image目錄

FACENET_DATA_PATH = os.path.join(DATA_PATH, "facenet","20180402-114759","20180402-114759.pb") #dacenet路徑

SVM_DATA_PATH = os.path.join(DATA_PATH, "serializer", "lfw_svm_classifier.pkl") #svm路徑
   
    
#----------載入MTCNN-----------#
    
emb_array, labels, labels_dict = lmtcnn.load_MTCNN(DATA_PATH)


#--------------保存facenet.pkl-------------#          
# 人臉特徵
emb_features_file = open(os.path.join(DATA_PATH+'/recognizer', 'lfw_emb_features.pkl'), 'wb')
pickle.dump(emb_array, emb_features_file)
emb_features_file.close()

# 矩陣
emb_lables_file = open(os.path.join(DATA_PATH+'/recognizer', 'lfw_emb_labels.pkl'), 'wb')
pickle.dump(labels, emb_lables_file)
emb_lables_file.close()

# user_ids
emb_lables_dict_file = open(os.path.join(DATA_PATH+'/recognizer', 'lfw_emb_labels_dict.pkl'), 'wb')
pickle.dump(labels_dict, emb_lables_dict_file)
emb_lables_dict_file.close()
    
#-------------------準備相關變數-----------------#
    
# 訓練/測試變數
X_train = []; y_train = []
X_test = []; y_test = []

# 保存己經有處理過的人臉label
processed = set()
    
path = DATA_PATH + '/recognizer'
    
emb_features, emb_labels, emb_labels_dict, emb_dict = pkl.load_face_pkl(path, DATA_PATH)

# 分割訓練資料集與驗證資料集
for (emb_feature, emb_label) in zip(emb_features, emb_labels):
    if emb_label in processed:
        X_train.append(emb_feature)
        y_train.append(emb_label)
    else:
        X_test.append(emb_feature)
        y_test.append(emb_label)
        processed.add(emb_label)

# 結果
print('X_train: {}, y_train: {}'.format(len(X_train), len(y_train)))
print('X_test: {}, y_test: {}'.format(len(X_test), len(y_test)))


#----------------訓練人臉分類器(SVM Classifier)-----------------#
#使用scikit-learn的SVM分類器來進行訓練。
#使用linearSvc來訓練

# 訓練分類器
print('Training classifier')
linearsvc_classifier = LinearSVC(C=1, multi_class='crammer_singer')

# 進行訓練
linearsvc_classifier.fit(X_train, y_train)

classifier_filename = SVM_DATA_PATH


class_names = []
for key in sorted(emb_labels_dict.keys()):
    class_names.append(emb_labels_dict[key].replace('_', ' '))


with open(classifier_filename, 'wb') as outfile:
    pickle.dump((linearsvc_classifier, class_names), outfile)

print('Saved classifier model to file "%s"' % classifier_filename)

face_id_name = ''

Origin: Number of classes: 25
Origin: Number of images: 344
Model filename: data/facenet/20180402-114759/20180402-114759.pb


100%|██████████| 688/688 [00:05<00:00, 125.73it/s]


人臉特徵數量: 344, shape: (344, 512), type: <class 'numpy.ndarray'>
人臉標籤數量: 344, type: <class 'list'>
人臉標籤種類: {}, type: {} 25 <class 'dict'>
X_train: 319, y_train: 319
X_test: 25, y_test: 25
Training classifier
Saved classifier model to file "data/serializer/lfw_svm_classifier.pkl"
